In [1]:
%%writefile /home/melinda_yang/pylib/plotting_funcs.py

def mkDZary(filename,tp=True):
    import numpy as np
    ##filename == name of file, including pD and ending
    ##if need to transpose array, then transpose
    myfile = open(filename,'r')
    myp2s = []
    badindex = -5
    for ind,line in enumerate(myfile): 
        x = line.split()    
        if ind == 0: 
            myp1s = x[1:]
            if 'Pops' in myp1s:
                badindex = myp1s.index('Pops')
                myp1s.remove('Pops')
            continue
        if badindex != -5: data = [float(i) for ind2,i in enumerate(x[1:]) if x!='nan' and ind2 != badindex]
        else: data = [float(i) for ind2,i in enumerate(x[1:]) if x!='nan']
        if ind == 1: myary = np.array(data)
        else: myary = np.vstack((myary,np.array(data)))
        myp2s.append(x[0])
    if 'tp' == True: myary = np.transpose(myary)    
    myfile.close()
    return myary,myp1s,myp2s

def runD(pop1,pop2,mytuples,pD,fh,subset,chosenindices,labelind,fixed=(0,0,0,0),xlims=(-0.1,0.15,0.01),
         fontsize=25,figsize=(50,50),offset=0.01,outpd='',f4mode=False,indfilename="None",badsnpfile=0):
    import plotting_funcs as pf
    import projmodules as pm
    from subprocess import Popen, PIPE

    ###
    if f4mode == True: lab='f4'
    else: lab='D'
    if outpd == '': newfile = open(pD+fh+'.'+lab+'.'+subset+".pop",'w')
    else: newfile = open(outpd+fh+'.'+lab+'.'+subset+".pop",'w')
    for mytup in mytuples: newfile.write('\t'.join([i for i in mytup]) + '\n')
    newfile.close()

    ##Make qpDstat files
    print 'Calculating D...'
    
    pm.Dparfile(pD,fh,subset,f4mode,indfilename,badsnpfile,outpd)
    
    if outpd=='': 
        outfile = open(pD+fh+'.'+lab+'.'+subset+".log",'w')
        mycommand = 'qpDstat -p '+pD+fh+'.'+lab+'.'+subset+'.par'
    else: 
        outfile = open(outpd+fh+'.'+lab+'.'+subset+".log",'w')
        mycommand = 'qpDstat -p '+outpd+fh+'.'+lab+'.'+subset+'.par'
    print 'Running command', mycommand
    
    varfile = Popen(mycommand.split(), stdout = outfile)
    varfile.communicate()
    outfile.close()

    print 'Making D Matrix'
    if outpd == '': mypD=pD
    else: mypD=outpd
    myres = pf.getDZ(mypD,fh+'.'+lab+'.'+subset)
    makeDmatrix(pop1,pop2,myres,mypD+fh+'.'+lab+'.'+subset+'.Zmat',
            fixed=fixed,
            chosenindices=chosenindices,
            DZsd=1)
    makeDmatrix(pop1,pop2,myres,mypD+fh+'.'+lab+'.'+subset+'.Dmat',
            fixed=fixed,
            chosenindices=chosenindices,
            DZsd=0)
    pf.makeDmatrix(pop1,pop2,myres,mypD+fh+'.'+lab+'.'+subset+'.Smat',
            fixed=fixed,
            chosenindices=chosenindices,
            DZsd=3)
    pf.makeDmatrix(pop1,pop2,myres,mypD+fh+'.'+lab+'.'+subset+'.SDmat',
            fixed=fixed,
            chosenindices=chosenindices,
            DZsd=4)
    pf.makeDmatrix(pop1,pop2,myres,mypD+fh+'.'+lab+'.'+subset+'.SD1mat',
            fixed=fixed,
            chosenindices=chosenindices,
            DZsd=5)
    pf.makeDmatrix(pop1,pop2,myres,mypD+fh+'.'+lab+'.'+subset+'.SD2mat',
            fixed=fixed,
            chosenindices=chosenindices,
            DZsd=6)
    pf.makeDmatrix(pop1,pop2,myres,mypD+fh+'.'+lab+'.'+subset+'.SEmat',
            fixed=fixed,
            chosenindices=chosenindices,
            DZsd=2)
    '''
    print 'Plotting D...'
    makeDploth(fh+'.D.'+subset,fh+'.D.'+subset,pD,
             xlims=xlims,
             fontsize=fontsize,
             figsize=figsize,
             offset=offset,
             labelind=labelind,
             out=fh+'.D.'+subset)
    '''
    return 'Done!'

def runD_limited(mytuples,pD,fh,subset,outpd=''):
    import plotting_funcs as pf
    import projmodules as pm
    from subprocess import Popen, PIPE

    ###
    newfile = open(pD+fh+'.D.'+subset+".pop",'w')
    for mytup in mytuples: newfile.write('\t'.join([i for i in mytup]) + '\n')
    newfile.close()

    ##Make qpDstat files
    #print 'Calculating D...'
    pm.Dparfile(pD+fh,subset)
    if outpd=='': outfile = open(pD+fh+'.D.'+subset+".log",'w')
    else: outfile = open(outpd+fh+'.D.'+subset+".log",'w')
    mycommand = 'qpDstat -p '+pD+fh+'.D.'+subset+'.par'
    #print 'Running command', mycommand
    
    varfile = Popen(mycommand.split(), stdout = outfile)
    varfile.communicate()
    outfile.close()

    #print 'Making D Matrix'
    myres = pf.getDZ(pD,fh+'.D.'+subset)
    
    return myres

def getDZ(projDirectory,Dfilelog):
    ##Get D statistic and Z score, sd and numSNPs considered
    myfile = open(projDirectory+Dfilelog+'.log','r')
    myres = {}
    for line in myfile:
        if line[:4] != 'resu': continue
        x = line.strip().split()
        D = float(x[5])
        Z = float(x[7])
        S1 = int(x[8])
        S2 = int(x[9])
        if Z != 0: sd = D/Z
        else: sd = 0
        if len(x) <= 10: numSNPs = int(x[-1])
        else: numSNPs = int(x[10])
        myres[(x[1],x[2],x[3],x[4])] = [D,Z,sd,numSNPs,S1-S2,S1,S2]
    return myres

def makeDmatrix(pop1,pop2,Dstats,newfilename,fixed=(0,0,0,0),chosenindices=(0,1),DZsd=0):
    ##pop1, pop2 = two populations to vary on matrix
    ##Dstats = D,Z,sd in dictionary, with tuple as key, format like in pf.getDZ
    ##newfile = newfilename to write matrix to, need directory!
    ##fixed=(0,0,0,0); place name of each popn, especially the fixed ones
    ##chosenindices=(0,1), choose which positions in tuple to vary over
    ##DZsd=0, Do you want a matrix of D (0), Z(1), or sd (2) or S (3)?
    ##makeDmatrix(ancients,ancients,myres,md.projDir1+fh+'.D.'+subset+'.mat',
    ##        fixed=(0,0,'Han','Mbuti'),
    ##        chosenindices=(0,1),
    ##        DZsd=1)
    import numpy as np
    newfile = open(newfilename,'w')
    newfile.write('Pops\t'+'\t'.join([i for i in pop2])+'\n')
    myary = np.zeros((len(pop1),len(pop2)))
    for ind1,i1 in enumerate(pop1):
        for ind2,i2 in enumerate(pop2):
            regfixed = list(fixed)
            regfixed[chosenindices[0]] = i1
            regfixed[chosenindices[1]] = i2
            regfixed = tuple(regfixed)
            p1,p2,p3,p4 = regfixed
            if regfixed in Dstats: myary[ind1,ind2] = Dstats[regfixed][DZsd]
            else: myary[ind1,ind2] = np.nan
    for row in range(myary.shape[0]):
        newfile.write(pop1[row]+'\t'+'\t'.join([str(i) for i in list(myary[row,])])+'\n')
    return 'Made file '+newfilename

def makeDploth(Dfilelog,targetfilename,projDirectory,out="NA",fontsize=40,figsize=(15,15),xlims=(-0.07,0.07,0.01),labelind=(0,1),offset=0.001):
    ##Dfilelog -- don't add suffix '.log', the qpDstat results
    ##targetfilename -- don't add suffix '.txt', the list of Dstats to display, with labels at end
    ##projDirectory--the folder path to all files!
    ##fontsize--integer, font size of everything on graph
    ##figsize--tuple of two integers, indicating figsize
    ##ylims--tuple of three integers indicating (ymin,ymax,ystep)
    ##makeDplot("UPA_all.041816.D.ALL","SGDP_SGDP_TY_Chimp_041816_1.EAs_v_NAs1",40,md.projDir1)
    import matplotlib
    matplotlib.use('Agg')
    import numpy as np
    import matplotlib.pyplot as plt
    import matplotlib.patches as mpatches
    import TY_maindata as md
    import pandas as pd

    targetfile = open(projDirectory+targetfilename+'.pop','r')
    targetpops = [tuple(line.strip().split()) for line in targetfile]

    targets = []
    for mytup in targetpops: 
        if mytup[-1] not in targets: targets.append(mytup[-1])
        else: continue
        
    ##Get D statistic and Z score
    myres = getDZ(projDirectory,Dfilelog)
    '''
    myfile = open(projDirectory+Dfilelog+'.log','r')
    myres = {}
    for line in myfile:
        if line[:2] != 're': continue
        x = line.strip().split()
        D = float(x[5])
        Z = float(x[7])
        if Z != 0: sd = D/Z
        else: sd = 0
        myres[(x[1],x[2],x[3],x[4])] = [D,Z,sd]
    '''
    cmap = plt.get_cmap('nipy_spectral')
    color_list = [cmap(i) for i in np.linspace(0,1,len(targets)+1)]
    
    ##Make lists of relevant D vals
    myDvals,myids,myZs,mycolors,mypops = [],[],[],[],[]
    for p1,p2,p3,p4,label in targetpops:
        myset = (p1,p2,p3,p4)
        mycolors.append(color_list[targets.index(label)])
        if (p1,p2,p3,p4) in myres:
            myDvals.append(myres[myset][0])
            myZs.append(myres[myset][1])
        elif (p2,p1,p3,p4) in myres:
            myDvals.append(-myres[(p2,p1,p3,p4)][0])
            myZs.append(-myres[(p2,p1,p3,p4)][1])
        myids.append('_'.join([myset[i] for i in labelind]))    
        ##myids.append(p1+'_'+p2)
    
    freq_series = pd.Series.from_array(myDvals)        
    fig = plt.figure(figsize=figsize)
    fontparam = {'fontsize':fontsize}
    bar_width = 0.5
    xticks = np.arange(xlims[0],xlims[1],xlims[2])
    plt.figure(figsize=(36, 24))
    ax = freq_series.plot(kind='barh',color=mycolors)
    ax.set_xlim(xticks[0],xticks[-1])
    ##ax.set_xticks(range(len(myDvals)+1))
    ax.set_xticks(xticks)
    ax.set_xticklabels(['%.2f' % i for i in xticks],fontsize=fontsize)
    ax.set_yticklabels(myids,fontsize=fontsize,rotation=0,minor=False)
    ax.set_title(targetfilename,fontparam)

    rects = ax.patches

    # Now make some labels
    labels = ["****Z=%.4f" % i if abs(i)>3 else "%1.1f" % i for i in myZs]
    count = 0
    for rect, label in zip(rects, labels):
        if myDvals[count] <= 0: offsetcorr = -offset
        else: offsetcorr = offset
        height = rect.get_width()
        ##print myDvals[count], offset, myDvals[count]+offset
        ax.text(myDvals[count]+offsetcorr, ## + rect.get_height()/2,
                rect.get_y() + 0.75, label, ##height + 0.005,label,
                fontsize=fontsize,
                ha='center',
                va='top',
                rotation=0)
        count += 1
    #Get artists and labels for legend and chose which ones to display
    handles, labels = ax.get_legend_handles_labels()
    display = (0,0,0)

    #Create custom artists
    myArtists = [plt.Line2D((0,1),(0,0),
                        color=color_list[targets.index(i)],
                        linewidth=8) for i in targets] 

    #Create legend from custom artist/label lists
    ax.legend(myArtists,
              targets,
              fontsize=fontsize,
              loc=4)
    if out == "NA": plt.savefig(projDirectory+Dfilelog+'.'+targetfilename+'.png',bbox_inches='tight')
    else: plt.savefig(projDirectory+out+'.png',bbox_inches='tight')
    return 'Made D plot!'


Writing /home/melinda_yang/pylib/plotting_funcs.py


In [1]:
def makeDplot(Dfilelog,targetfilename,projDirectory,fontsize=40,figsize=(50,15),ylims=(-0.07,0.07,0.01),labelind=(0,1)):
    ##Dfilelog -- don't add suffix '.log', the qpDstat results
    ##targetfilename -- don't add suffix '.txt', the list of Dstats to display, with labels at end
    ##projDirectory--the folder path to all files!
    ##fontsize--integer, font size of everything on graph
    ##figsize--tuple of two integers, indicating figsize
    ##ylims--tuple of three integers indicating (ymin,ymax,ystep)
    ##makeDplot("UPA_all.041816.D.ALL","SGDP_SGDP_TY_Chimp_041816_1.EAs_v_NAs1",40,md.projDir1)
    import matplotlib
    matplotlib.use('Agg')
    import numpy as np
    import matplotlib.pyplot as plt
    import matplotlib.patches as mpatches
    import TY_maindata as md
    import pandas as pd

    targetfile = open(projDirectory+targetfilename+'.pop','r')
    targetpops = [tuple(line.strip().split()) for line in targetfile]

    targets = []
    for mytup in targetpops: 
        if mytup[-1] not in targets: targets.append(mytup[-1])
        else: continue
        
    ##Get D statistic and Z score
    myfile = open(projDirectory+Dfilelog+'.log','r')
    myres = {}
    for line in myfile:
        if line[:2] != 're': continue
        x = line.strip().split()
        D = float(x[5])
        Z = float(x[7])
        if Z != 0: sd = D/Z
        else: sd = 0
        myres[(x[1],x[2],x[3],x[4])] = [D,Z,sd]

    cmap = plt.get_cmap('nipy_spectral')
    color_list = [cmap(i) for i in np.linspace(0,1,len(targets)+1)]
    
    ##Make lists of relevant D vals
    myDvals,myids,myZs,mycolors,mypops = [],[],[],[],[]
    for p1,p2,p3,p4,label in targetpops:
        myset = (p1,p2,p3,p4)
        mycolors.append(color_list[targets.index(label)])
        if (p1,p2,p3,p4) in myres:
            myDvals.append(myres[myset][0])
            myZs.append(myres[myset][1])
        else:
            myDvals.append(-myres[(p2,p1,p3,p4)][0])
            myZs.append(-myres[(p2,p1,p3,p4)][1])
        myids.append('_'.join([myset[i] for i in labelind]))    
        ##myids.append(p1+'_'+p2)
    
    freq_series = pd.Series.from_array(myDvals)        
    fig = plt.figure(figsize=figsize)
    fontparam = {'fontsize':fontsize}
    bar_width = 0.5
    yticks = np.arange(ylims[0],ylims[1],ylims[2])
    ##plt.figure(figsize=(36, 24))
    ax = freq_series.plot(kind='bar',color=mycolors)
    ax.set_ylim(yticks[0],yticks[-1])
    ##ax.set_xticks(range(len(myDvals)+1))
    ax.set_yticks(yticks)
    ax.set_yticklabels(['%.2f' % i for i in yticks],fontsize=fontsize)
    ax.set_xticklabels(myids,fontsize=fontsize,rotation=90,minor=False)
    ax.set_title(targetfilename,fontparam)

    rects = ax.patches

    # Now make some labels
    labels = ["****Z=%.4f" % i if abs(i)>3 else "%1.1f" % i for i in myZs]

    for rect, label in zip(rects, labels):
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width()/2,
                height + 0.005,label,
                fontsize=fontsize,
                ha='center',
                va='bottom',
                rotation=90)

    #Get artists and labels for legend and chose which ones to display
    handles, labels = ax.get_legend_handles_labels()
    display = (0,0,0)

    #Create custom artists
    myArtists = [plt.Line2D((0,1),(0,0),
                        color=color_list[targets.index(i)],
                        linewidth=8) for i in targets] 

    #Create legend from custom artist/label lists
    ax.legend(myArtists,
              targets,
              fontsize=fontsize,
              loc=4)

    plt.savefig(projDirectory+Dfilelog+'.'+targetfilename+'.png',bbox_inches='tight')
    return 'Made D plot!'

def add_range_annotation(ax, start, end, txt_str, y_height=.5, txt_kwargs=None, arrow_kwargs=None):
    """
    Adds horizontal arrow annotation with text in the middle

    Parameters
    ----------
    ax : matplotlib.Axes
        The axes to draw to

    start : float
        start of line

    end : float
        end of line

    txt_str : string
        The text to add

    y_height : float
        The height of the line

    txt_kwargs : dict or None
        Extra kwargs to pass to the text

    arrow_kwargs : dict or None
        Extra kwargs to pass to the annotate

    Returns
    -------
    tuple
        (annotation, text)
    """

    if txt_kwargs is None:
        txt_kwargs = {}
    if arrow_kwargs is None:
        # default to your arrowprops
        arrow_kwargs = {'arrowprops':dict(arrowstyle="<->",
                            connectionstyle="bar",
                            ec="k",
                            shrinkA=5, shrinkB=5,
                            )}

    trans = ax.get_xaxis_transform()

    ann = ax.annotate('', xy=(start, y_height),
                        xytext=(end, y_height),
                        transform=trans,
                        **arrow_kwargs)
    txt = ax.text((start + end) / 2,
                  y_height + .05,
                  txt_str,
                  **txt_kwargs)


    if plt.isinteractive():
        plt.draw()
    return ann, txt

In [ ]:
%%writefile /home/mel_yang/projects/TY/makef3plot.py
##%matplotlib inline

import matplotlib
matplotlib.use('Agg')
import numpy as np
import matplotlib.pyplot as plt
import TY_maindata as md

fh = 'UPA_all.041816'
p1,p2,p3 = ('TY','Anc','Mbuti')
subset = '%s_%s_%s' % (p1,p2,p3)

myfile = open(md.projDir1+fh+'.f3.'+subset+'.log','r')
myres = {}
for line in myfile:
    if line[:2] != ' r': continue
    x = line.strip().split()
    myres[x[2]] = float(x[4])
    
mygroups = md.fu2016_popngroups
cmap = plt.get_cmap('nipy_spectral')
color_list = [cmap(i) for i in np.linspace(0,1,len(mygroups.keys())+1)]

fig = plt.figure(figsize=(50,15))
fontparam = {'fontsize':60}
bar_width = 0.8
subplots = [221,222,223,224]
subplots = 111
yticks = np.arange(0.2,0.26,0.01)
myf3vals,myids,myperiod,mycolors = [],[],[],[]
for ind,period in enumerate(['EUP','MUP','LUP','EN']):
    for indiv in mygroups[period]:
        myf3vals.append(myres[indiv])
        myids.append(indiv)
        myperiod.append(period)
        mycolors.append(color_list[ind])
ax = fig.add_subplot(subplots)
index = np.arange(len(myf3vals))
ax.bar(index,myf3vals,bar_width,color=mycolors,align='center')
ax.set_title("f3(TY,Fuetal 2016, Mbuti)",fontparam)
ax.set_ylim(yticks[0],yticks[-1])
ax.set_xticks(range(len(myf3vals)))
ax.set_yticks(yticks)
ax.set_yticklabels(yticks,fontsize=45)
ax.set_xticklabels(myids,fontsize=45,rotation=90,minor=False)
##ax.legend(range(len(mygroups.keys())),mygroups.keys(),fontsize=20) 

fig.savefig(md.projDir1+fh+'.f3.'+subset+'.png',bbox_inches='tight')
plt.close(fig)


In [ ]:
%%writefile /home/mel_yang/projects/TY/makeDplot_bycountry.py
import matplotlib
matplotlib.use('Agg')
##%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import TY_maindata as md
import pandas as pd

fh = 'UPA_all.041816'
p1,p2,p3,p4 = ('SGDP','SGDP','TY','ChimpMbuti')
subset = '%s_%s_%s_%s' % (p1,p2,p3,p4)
mycountries = ['Cambodia','Thailand','PapuaNewGuinea','New Zealand','Yemen',
                'Pakistan','Taiwan','Bangaladesh','Nepal','Korea','China',
                'Vietnam','Myanmar','Japan','France','England']
mypair = ['China','Taiwan']
pair1 = md.sgdpc_country[mypair[0]]
pair2 = md.sgdpc_country[mypair[1]]
within1 = [(i,j) for i in pair1 for j in pair1 if i != j]
within2 = [(i,j) for i in pair2 for j in pair2 if i != j]
between = [(i,j) for i in pair1 for j in pair2 if i != j]
pop3,pop4 = p3,'Chimp'

myfile = open(md.projDir1+fh+'.D.'+subset+'.log','r')
myres = {}
for line in myfile:
    if line[:2] != 're': continue
    x = line.strip().split()
    D = float(x[5])
    Z = float(x[7])
    sd = D/Z
    myres[(x[1],x[2],x[3],x[4])] = [D,Z,sd]

fig = plt.figure(figsize=(20,15))
fontparam = {'fontsize':20}
fontsize=20
bar_width = 0.5
subplots = 111
yticks = np.arange(-0.05,0.1,0.05)

myDvals,myids,myZs,mycolors,mypops = [],[],[],[],[]
mycols = ['red','blue','brown']
for ind,mygroup in enumerate([within1,between,within2]):
    for pop1,pop2 in mygroup:
        myset = (pop1,pop2,pop3,pop4)
        if (pop1,pop2) in mypops: continue
        elif (pop2,pop1) in mypops: continue
        else: mypops.append((pop1,pop2))
        if myset in myres: 
            myDvals.append(myres[myset][0])
            myZs.append(myres[myset][1])
        else: 
            myDvals.append(-myres[(pop2,pop1,pop3,pop4)][0])
            myZs.append(-myres[(pop2,pop1,pop3,pop4)][1])
        myids.append(pop1+'_'+pop2)
        mycolors.append(mycols[ind])

freq_series = pd.Series.from_array(myDvals)
index = np.arange(len(myDvals))

plt.figure(figsize=(36, 24))
ax = freq_series.plot(kind='bar',color=mycolors)
ax.set_ylim(yticks[0],yticks[-1])
##ax.set_xticks(range(len(myDvals)+1))
ax.set_yticks(yticks)
ax.set_yticklabels(yticks,fontsize=fontsize)
ax.set_xticklabels(myids,fontsize=fontsize,rotation=90,minor=False)
ax.set_title("D(SGDP_C,SGDP_C, TY, Chimp)",fontparam)

rects = ax.patches

# Now make some labels
labels = ["Z=%.4f" % i if abs(i)>3 else "" for i in myZs]

for rect, label in zip(rects, labels):
    height = rect.get_height()
    ax.text(rect.get_x() + rect.get_width()/2,
            height + 0.005,label,
            fontsize=fontsize,
            ha='center',
            va='bottom',
            rotation=90)

plt.savefig(md.projDir1+fh+'.D.'+subset+'.png')
##plt.close(fig)


In [ ]:
%%writefile /home/mel_yang/projects/TY/makeDplot_bylistedpairs.py
##Input tuples of four elements and the X-label for the D plot
##python makeDplot_bylistedpairs.py Dlogfilename(no log) outgroup targetfilename targetDindex(1,2,3,4)
##Assumes all in same folder
##python makeDplot_bylistedpairs.py UPA_all.041816.D.ALL Chimp TY_Anc_Arc_Chimp_selected_041816data 2


import matplotlib
matplotlib.use('Agg')
##%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import TY_maindata as md
import pandas as pd
import sys

Dfilelog = sys.argv[1]
outgroup = sys.argv[2]
targetfilename = sys.argv[3]
targetDindex = int(sys.argv[4])

##fh = 'UPA_all.041816'
##subset = 'ALL'
##outgroup = 'Chimp'
##targets = ['Loschbour','LaBrana1','Kostenki14','GoyetQ116-1','Vestonice16','Villabruna']
##targets = ['Altai','Denisovan','Mezmaiskaya','Vindija']
##targetfilename = "Anc_SGDP_TY_Chimp_selection_041816data"
##targetfilename = "TY_SGDP_Arc_Chimp_selection_041816data"
##targetfilename = "TY_Anc_Arc_Chimp_selected_041816data"
##targetDindex = 3
##targetDindex = 2
targetfile = open(targetfilename+'.txt','r')
targetpops = [tuple(line.strip().split()) for line in targetfile]
targetpops = [i for i in targetpops if outgroup in i]
targets = []
for mytup in targetpops: 
    if mytup[targetDindex-1] not in targets: targets.append(mytup[targetDindex-1])
    else: continue
##Get D statistic and Z score
myfile = open(Dfilelog+'.log','r')
myres = {}
for line in myfile:
    if line[:2] != 're': continue
    x = line.strip().split()
    D = float(x[5])
    Z = float(x[7])
    sd = D/Z
    myres[(x[1],x[2],x[3],x[4])] = [D,Z,sd]

cmap = plt.get_cmap('nipy_spectral')
color_list = [cmap(i) for i in np.linspace(0,1,len(targets)+1)]
    
##Make lists of relevant D vals
myDvals,myids,myZs,mycolors,mypops = [],[],[],[],[]
for p1,p2,p3,p4 in targetpops:
    myset = (p1,p2,p3,p4)
    mycolors.append(color_list[targets.index(myset[targetDindex-1])])
    if (p1,p2,p3,p4) in myres:
        myDvals.append(myres[myset][0])
        myZs.append(myres[myset][1])
    else:
        myDvals.append(-myres[(p2,p1,p3,p4)][0])
        myZs.append(-myres[(p2,p1,p3,p4)][1])
    myids.append(p1+'_'+p2)
    
freq_series = pd.Series.from_array(myDvals)        
fig = plt.figure(figsize=(20,15))
fontparam = {'fontsize':40}
fontsize=40
bar_width = 0.5
yticks = np.arange(-0.5,0.5,0.05)
plt.figure(figsize=(36, 24))
ax = freq_series.plot(kind='bar',color=mycolors)
ax.set_ylim(yticks[0],yticks[-1])
##ax.set_xticks(range(len(myDvals)+1))
ax.set_yticks(yticks)
ax.set_yticklabels(['%.2f' % i for i in yticks],fontsize=fontsize)
ax.set_xticklabels(myids,fontsize=fontsize,rotation=90,minor=False)
ax.set_title(targetfilename,fontparam)

rects = ax.patches

# Now make some labels
labels = ["****Z=%.4f" % i if abs(i)>3 else "%1.1f" % i for i in myZs]

for rect, label in zip(rects, labels):
    height = rect.get_height()
    ax.text(rect.get_x() + rect.get_width()/2,
            height + 0.005,label,
            fontsize=fontsize,
            ha='center',
            va='bottom',
            rotation=90)

#Get artists and labels for legend and chose which ones to display
handles, labels = ax.get_legend_handles_labels()
display = (0,0,0)

#Create custom artists
myArtists = [plt.Line2D((0,1),(0,0),
                        color=color_list[targets.index(i)],
                        linewidth=8) for i in targets] 

#Create legend from custom artist/label lists
ax.legend(myArtists,
          targets,
          fontsize=fontsize,
          loc=4)

plt.savefig(Dfilelog+'.'+targetfilename+'.out'+outgroup+'.png',bbox_inches='tight')
